In [ ]:
!pip install opencv-contrib-python
!pip install imutils
!pip install easyocr

In [25]:
import cv2
import imutils
import pandas as pd
import numpy as np
from easyocr import Reader
from matplotlib import pyplot as plt

In [26]:
def get_plate(img_name):
  image = cv2.imread(img_name)
  image = imutils.resize(image, width=300 )
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17) 
  edged = cv2.Canny(gray_image, 30, 200) 
  cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  image1=image.copy()
  cv2.drawContours(image1,cnts,-1,(0,255,0),3)
  cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
  screenCnt = None
  image2 = image.copy()
  cv2.drawContours(image2,cnts,-1,(0,255,0),3)
  for c in cnts:
          perimeter = cv2.arcLength(c, True)
          approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
          if len(approx) == 4: 
                  screenCnt = approx
                  x,y,w,h = cv2.boundingRect(c) 
                  new_img=image[y:y+h,x:x+w]
                  cv2.imwrite('./'+'temp.png',new_img)
                  break
  cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 3)

In [27]:
def read_plate(img_name):
  license_plate=cv2.imread(img_name)
  reader = Reader(['en'])
  detection = reader.readtext(license_plate,paragraph="False")
  res=""
  for i in detection:
    res+=i[1]
  return res

In [28]:
def detect_color(img_name):
  plate_image=cv2.imread(img_name)
  counter=0
  b=[]
  g=[]
  r=[]
  for x in np.nditer(plate_image):
    if counter==0:
      b.append(int(x))
      counter+=1
    elif counter==1:
      g.append(int(x))
      counter+=1
    else:
      r.append(int(x))
      counter=0
  #DETERMINE IF RED COLOR IS PRESENT
  c0=0
  c1=0
  for i in r:
    if i<55:
      c0+=1
    elif i>200:
      c1+=1
  if c0>c1:
    red=False
  else:
    red=True

  #DETERMINE IF BLUE COLOR IS PRESENT
  c0=0
  c1=0
  for i in b:
    if i<55:
      c0+=1
    elif i>200:
      c1+=1
  if c0>c1:
    blue=False
  else:
    blue=True

  #DETERMINE IF GREEN COLOR IS PRESENT
  c0=0
  c1=0
  for i in g:
    if i<127:
      c0+=1
    elif i>128:
      c1+=1
  if c0>c1:
    green=False
  else:
    green=True

  if red and green and blue:
    return("WHITE")
  elif red and green:
    return("YELLOW")
  elif red:
    return("RED")
  elif green:
    return("GREEN")
  elif blue:
    return("BLUE")
  else:
    return("BLACK")

In [29]:
data=pd.read_csv('dataset.csv')
#print(data)

In [30]:
contour_detection_success=0
contour_detection=0
plate_detection_success=0
plate_detection=0
color_detection_success=0
color_detection=0

In [31]:
for i in range(len(data['img'])):
  img_name='images/'+data['img'][i]
  if data['img'][i].startswith("car"):
    try:
      contour_detection+=1
      flag=True
      get_plate(img_name)
    except Exception as e:
      flag=False
    if flag:
      contour_detection_success+=1
      plate_detection+=1
      color_detection+=1
      plate=read_plate('temp.png')
      check=0
      for x in data['plate'][i]:
        if x in plate.upper():
          check+=1
      if check/len(data['plate'][i])>=0.75:
        plate_detection_success+=1
      color=detect_color('temp.png')
      if color==data['color'][i]:
        color_detection_success+=1
  else:
    plate_detection+=1
    color_detection+=1
    plate=read_plate(img_name)
    check=0
    for x in data['plate'][i]:
        if x in plate.upper():
          check+=1
    if check/len(data['plate'][i])>=0.75:
        plate_detection_success+=1
    color=detect_color(img_name)
    if color==data['color'][i]:
        color_detection_success+=1

In [ ]:
print("Accuracy of Contour Detection is: %.2f"%(contour_detection_success/contour_detection))
print("Accuracy of Plate Reading is: %.2f"%(plate_detection_success/plate_detection))
print("Accuracy of Color Detection is: %.2f"%(color_detection_success/color_detection))

In [ ]:
print("Overall Accuracy is: %.2f"%((contour_detection_success+plate_detection_success+color_detection_success)/(contour_detection+plate_detection+color_detection)))